# Solução de sistemas de equações algébricas

Um problema muito comum é encontrar o zero de funções, ou seja, encontrar $x$ de modo que

$$
f(x) = 0
$$

A dificuldade deste tipo de problema é que pode não haver nenhuma solução, uma ou mais de uma! 

São raros os casos onde existe uma solução explícita. Iteração se torna necessário. 

Existem várias estratégias para se encontrar a raíz. Uma abordagem bem genérica é encontrar uma outra função $g(x)$ de modo que

$$
x_{n+1} = g(x_n)
$$

convirja para a solução do problema original. O teorema do ponto fixo mostra em que condições este processo converge.

<!-- TEASER_END -->

## Método da bisecção

Caso saibamos que existe uma raíz em um intervalo, podemos ir sistematicamente dividindo o domínio em dois até chegar à solução

In [ ]:
function bisection(a, b, f, tol=1e-10, maxiter=1000)
    fa = f(a)
    fb = f(b)
    if fa==0.0
        return a,0
    elseif fb==0.0
        return fb,0
    elseif fa*fb > 0
        error("f(a) e f(b) devem ter sinais diferentes!")
    end
    p = 0.0
    for i in 1:maxiter
        dx = (b-a) / 2
        p = a + dx
        fp = f(p)
        if fp==0 || dx < tol
            return p,i
        end
        if fa*fp > 0
            a = p
            fa = fp
        else
            b = p
            fb = fp
        end
    end
    
    return p, maxiter
    
        
end


In [ ]:
g(x) = x*x-2.0
dg(x) = 2x

In [ ]:
bisection(0.0, 2.0, g)

In [ ]:
function plotbisection(a, b, f, tol=1e-10, maxiter=1000)
    fa = f(a)
    fb = f(b)
    if fa==0.0
        return a,0
    elseif fb==0.0
        return fb,0
    elseif fa*fb > 0
        error("f(a) e f(b) devem ter sinais diferentes!")
    end
    
    xx = range(a, b, length=201)
    plot(xx, f.(xx))
    axvline(x=a)
    axvline(x=b)
    p = 0.0
    for i in 1:maxiter
        dx = (b-a) / 2
        p = a + dx
        axvline(x=p)
        fp = f(p)
        if fp==0 || dx < tol
            return p,i
        end
        if fa*fp > 0
            a = p
            fa = fp
        else
            b = p
            fb = fp
        end
    end
    
    return p, maxiter
    
    
end

In [ ]:
plotbisection(0.0, 2.0, g)

## Método de Newton-Raphson

Este é o método básico para a solução de uma equação algébrica. A idéia é expandir a função em série de Taylor em torno de um chute inicial. Mantendo apenas os dois primeiros termos da expansão, 

$$
f(x + \Delta x) = 0 \:\Longrightarrow\: f(x+\Delta x) \approx f(x) + \Delta x\cdot f'(x) = 0 \:\Longrightarrow\: \Delta x = -\frac{f(x)}{f'(x)}
$$

espera-se chegar a uma estimativa melhor da solução. Este processo pode ser repetido até que a solução convirja. Este método já foi analisado no problema 5 da aula 7.

Repare que este método vai ter problema se na raíz

$$
f'(x) = 0
$$

In [ ]:
using ForwardDiff
using PyPlot

In [ ]:
function newtonraphson(f, df, x0, err=1e-10, maxiter=100)
    niter = 0
    for i = 1:maxiter
        dx = f(x0) / df(x0)
        x0 -= dx
        niter = i
        if abs(dx) < err
            break
        end
    end
    
    return x0, niter
end


In [ ]:
newtonraphson(x->x^2-2.0, x->2x, 2)

In [ ]:
newtonraphson(x->x^2-0.0, x->2x, 2)


In [ ]:
function plotnewton(f, df, x0, a, b, err=1e-5, maxiter=100)
    nn = 200
    xx = range(a,b, length=nn)
    yy = f.(xx)
    
    plot(xx, yy)
    axhline("k")
    iter = 0
    for i = 1:maxiter
        y = f(x0)
        dy = df(x0)
        dx = y / dy
        plot([x0, x0], [0.0, y], "k:")
        plot([x0, x0-dx], [y, 0.0], "k-")
        x0 = x0 - dx
        if abs(dx) < err
            return x0, i
        end
    end
        
        
end

In [ ]:
plotnewton(x->x^2-2.0, x->2x, 2.0, 1.0, 2.0)

In [ ]:
plotnewton(x->x^2, x->2x, 1.0, -0.2, 1)

In [ ]:
plotnewton(x->x^4, x->4x^3, 1.0, -0.2, 1)

In [ ]:
plotnewton(x->x^5, x->5x^4, 1.0, -1, 1)

In [ ]:
function newtonraphson2(f, x0, err=1e-10, maxiter=100)
    niter = 0
    for i = 1:maxiter
        dx = f(x0) / ForwardDiff.derivative(f,x0)
        x0 -= dx
        niter = i
        if abs(dx) < err
            break
        end
    end
    
    return x0, niter
    

end

## Método da secante

Uma dificuldade do método de Newton-Raphson é a necessidade de calcular derivadas. Isto já não é um problema se diferenciação automática é utilizada. Mas o custo computacional pode ser alto. 

Outra possibilidade é usar derivadas numéricas. Usando uma derivada numérica simples e incorporando isso diretamente no algorítmo, chega-se ao método da secante.

Dado dois chutes iniciais $x_0$ e $x_1$, existe uma reta que passa por ambos os pontos:



In [ ]:
function secant(f, x0, x1, tol=1e-10, maxiter=100)
    f0 = f(x0)
    f1 = f(x1)
    
    for i in 1:maxiter
        a = (f1-f0) / (x1-x0)
        b = f0 - a*x0
        
        x0 = x1
        f0 = f1
        x1 = -b / a
        f1 = f(x1)
        if abs(f1) < tol
            return x1, i
        end
    end
    error("Não convergiu!")
end

In [ ]:
secant(x->x^2-2.0, 2.5, 2.0)

In [ ]:
function plotsecant(f, xl, xr, x0, x1, tol=1e-10, maxiter=100)
    f0 = f(x0)
    f1 = f(x1)
    xx = range(xl, xr, length=200)
    
    plot(xx, f.(xx), "k-")
    axhline(0)
    plot([x0, x0], [0, f0], "k:")
    plot([x1, x1], [0, f1], "k:")
    
    for i in 1:maxiter
        a = (f1-f0) / (x1-x0)
        b = f0 - a*x0
        
        
        xn = -b/a
        
        plot([x0, xn], [f0, 0], "k-")
        
        x0 = x1
        f0 = f1
        x1 = xn
        f1 = f(x1)
        plot([x1, x1], [0, f1], "k:")
        if abs(f1) < tol
            return x1, i
        end
    end
    error("Não convergiu!")

end

In [ ]:
plotsecant(x->1-(x+1.5)^2, -0.5, 3.0, 2.5, 2.0)

# Sistema de equações não lineares

O problema é o mesmo mas agora queremos resolver

$$
f_1(x_1, x_2, \ldots, x_n) = 0\\
f_2(x_1, x_2, \ldots, x_n) = 0\\
\vdots\\
f_n(x_1, x_2, \ldots, x_n) = 0\\
$$

Newton Raphson pode ser prontamente generalizado para sistemas:

$$
f_i\left(x_1+\delta x_1, x_2 + \delta x_2, \ldots, x_n + \delta x_n\right) \approx f_i(x_1, x_2, \ldots, x_n) + \delta x_1 \frac{\partial f_i}{\partial x_1} + \delta x_2 \frac{\partial f_i}{\partial x_2} + \ldots + \delta x_n \frac{\partial f_i}{\partial x_n} = 0
$$

Portanto 

$$
\left[J\right] \cdot \left\{ \delta x\right\} = -\left\{f\right\}
$$

onde $[J]$ é a matriz Jacobiana e vale

$$
J_{i,k} = \left.\frac{\partial f_i}{\partial x_k}\right|_{x_1, x_2, \ldots, x_n}
$$

Assim o algorítmo é:

 1. Chute inicial $x_1^0, x_2^0, \ldots, x_n^0$
 2. Calcular a matriz Jacobiana
 3. Resolver o sistema linear $\left[J\right] \cdot \left\{ \delta x\right\} = -\left\{f\right\}$
 4. Calcular $x^1_i = x_i^0 + \delta x_i$
 5. Repetir até convergir
 

In [ ]:
function newtonraphson(n, f!, J!, xinit::AbstractVector, tol=1e-10, maxiter=100)
        
    x0 = [x for x in xinit]
    
    J = zeros(n,n)
    y = zeros(n)
    for i in 1:maxiter
        f!(y, x0)
        y .= .- y
        
        if maximum(abs, y) < tol
            return x0, i
        end
        
        J!(J, x0)
        
        δx = J\y
        
        x0 .= x0 .+ δx
            
        
    end
    error("Não convergiu")
end


### Exemplo


In [ ]:
function f!(y, x)
    y[1] = 3x[1] - cos(x[2]*x[3]) - 0.5
    y[2] = x[1]^2 - 81*(x[2] + 0.1)^2 + sin(x[3]) + 1.06
    y[3] = exp(-x[1]*x[2]) + 20x[3] + (10π-3)/3
end

function J!(J, x)
    J[1,1] = 3.0
    J[1,2] = x[3]*sin(x[2]*x[3])
    J[1,3] = x[2]*sin(x[2]*x[3])
    
    J[2,1] = 2x[1]
    J[2,2] = -162*x[2] - 16;2
    J[2,3] = cos(x[3])
    
    J[3,1] = -x[2] * exp(-x[1]*x[2])
    J[3,2] = -x[1] * exp(-x[1]*x[2])
    J[3,3] = 20.0
    
end



In [ ]:
newtonraphson(3, f!, J!, [0.1, 0.1, -0.1])

Podemos usar diferenciação automática

# Zero de uma função como um problema de otimização

Encontrar a raíz de um equação não linear pode ser reescrito como um problema de otimização. 
Se x é uma raíz de 

$$
f(x) = 0
$$

então x também é o ponto que mnimiza a seguinte fução:

$$
g(x) = \left[ f(x) \right]^2
$$

Achar o mínimo de g(x) é o mesmo que achar o zero de f(x)

**Steepest Descent**


# Pacotes Julia interessantes

Julia possui uma diversidade de pacotes para resolver equações e sistemas de equações não lineares.

 * <https://github.com/JuliaMath/Roots.jl>: implementa diversos métodos para resolver equações não lineares.
 * <https://github.com/JuliaIntervals/IntervalRootFinding.jl> é um pacote muito interessantes que permite encontrar *todas* as raízed de uma equação ou sistema em um intervalo
 * <https://github.com/JuliaNLSolvers/NLsolve.jl>: implementa métodos para resolver sistemas de equações não lineares
 * <https://github.com/JuliaMath/Polynomials.jl>: Este nosso velho amigo permite encontrar todas as raízes de um polinômio
 * <https://github.com/giordano/PolynomialRoots.jl>: Pacote para encontrar *todas* as raízes de polinômios
 * <https://github.com/JuliaApproximation/ApproxFun.jl> também permite encontrar as raízes de aproximações

# Problemas

## Problema 1: Método da secante para sistemas de equações não lineares

Implemente o método da secante para sistemas de equações não lineares e resolva o problema do exemplo acima


## Problema 2: Fractais de Newton

Ao se usar o método de Newton para se resolver uma equação que possui mais de uma solução, dependendo do valor inicial o método irá convergir para alguma das soluções. 

O senso comum diz que haverá uma fronteira dividindo regiões que convregem para diferentes soluções. Mas a realidade é muito mais interessante.

Os fractais de Newton são construídos aplicando o método de Newton a polinômios de grau qualquer *em um domínio complexo*. A wikipedia (<https://en.wikipedia.org/wiki/Newton_fractal>) mostra alguns exemplos de fractais de Newton.

O problema é fazer uma função que desenhe os fractais de Newton para um polinômio qualquer.

(O programa XaoS <http://matek.hu/xaos/doku.php> permite visualizar diversos tipos de fractais)